In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import BaseCrossValidator
from sklearn.linear_model import Ridge
import time
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor

In [2]:
df = pd.read_excel('Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833.0,12,0.21,2,22.0,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800.0,12,0.21,2,22.0,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826.0,12,0.21,2,22.0,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831.0,12,0.21,2,22.0,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834.0,12,0.21,2,22.0,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676.0,36,0.21,2,41.0,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683.0,36,0.21,2,42.0,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340909,50151.250000,77962.397727,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846.0,36,0.21,2,42.0,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694118,47515.850980,73948.509804,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876.0,36,0.21,2,42.0,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943182,42468.990341,65994.485795,3.789568e+07,3761.343750


In [3]:
df.replace('#DIV/0!',np.nan,inplace=True)
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37527 entries, 0 to 37527
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Backhoe      37527 non-null  int64  
 1   HP(watt)     37527 non-null  float64
 2   Norm_MAP     37527 non-null  float64
 3   RPM          37527 non-null  float64
 4   Age          37527 non-null  int64  
 5   Load_Factor  37527 non-null  float64
 6   Engine_Tier  37527 non-null  int64  
 7   TEMP[C]      37527 non-null  float64
 8   Fuel[g/s]    37527 non-null  float64
 9   NOx[g/s]     37527 non-null  float64
 10  HC[g/s]      37527 non-null  float64
 11  CO[g/s]      37527 non-null  float64
 12  CO2[g/s]     37527 non-null  float64
 13  PM[mg/s]     37527 non-null  float64
 14  Nox (g/l)    37527 non-null  float64
 15  HC (g/l)     37527 non-null  float64
 16  CO (g/l)     37527 non-null  float64
 17  CO2 (g/l)    37527 non-null  float64
 18  PM (g/l)     37527 non-null  float64
 19  Fuel

In [5]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['HC[g/s]']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(26268, 7)
(26268,)
(11259, 7)
(11259,)


In [8]:
rf = BaggingRegressor()

In [9]:
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

In [10]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

rmse_score = rmse(y_test, y_pred)
mape_score = mape(y_test, y_pred)

print(f'RMSE: {rmse_score}')
print(f"MAPE: {mape_score:.2f}%")

RMSE: 0.0015679911215357065
MAPE: 12576110376.56%


GRID

In [11]:
params = {
    'n_estimators': range(10, 500),
    'max_samples': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'max_features': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'bootstrap' : [True, False],
    'bootstrap_features': [True, False],
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error)

In [12]:
grid = GridSearchCV(
    rf,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE', 
    cv=2,  
    verbose=1,
    n_jobs=-1
)

start_time = time.time()
gbr_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 2 folds for each of 70560 candidates, totalling 141120 fits
Best: 0.001892 using {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.5, 'max_samples': 1.0, 'n_estimators': 17}
Best RMSE: 0.0018921997321694824
Best MAPE: 12349655737.147375
Execution time: 63418.121943473816 s


In [13]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 0.0019891718648100683
MAPE: 12626272750.16%


In [14]:
pd.DataFrame(grid.cv_results_).to_excel('BR_GS_unscal_HC.xlsx', index=False)

random search

In [15]:
rs = RandomizedSearchCV(
    rf,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    n_iter=100,
    cv=3,  
    verbose=1,
    n_jobs=-1
)

start_time = time.time()
rs_result = rs.fit(X_train,y_train)

print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best: 0.001762 using {'n_estimators': 473, 'max_samples': 0.5, 'max_features': 0.5, 'bootstrap_features': True, 'bootstrap': False}
Best RMSE: 0.0017616675845715272
Best MAPE: 11458719920.918486
Execution time: 167.4990906715393 s


In [16]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 0.0018077474336679344
MAPE: 12396792283.98%


In [17]:
pd.DataFrame(rs.cv_results_).to_excel('BR_RS_unscal_HC.xlsx', index=False)

Bayessian

In [18]:
bootstraps = [True, False]
bootstrap_feature = [True, False]

def br_regression_cv(n_estimators, max_samples, max_features, bootstrap, bootstrap_features):
    rf = BaggingRegressor(
        n_estimators=int(n_estimators),
        max_samples=float(max_samples),
        max_features=float(max_features),
        bootstrap=bootstraps[int(round(bootstrap))],  
        bootstrap_features=bootstrap_feature[int(round(bootstrap_features))]
    )
    scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse 

In [19]:
bayes_opt = BayesianOptimization(
    br_regression_cv,
    {
        'n_estimators': (10, 500),
        'max_samples': (0.5, 1),
        'max_features': (0.5, 1),
        'bootstrap': (0,1),
        'bootstrap_features':(0,1)
    }
)

In [20]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | bootstrap | bootst... | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------------------------------
| 1         | -0.001474 | 0.364     | 0.8332    | 0.8831    | 0.7595    | 107.9     |
| 2         | -0.001666 | 0.6121    | 0.01138   | 0.6867    | 0.6238    | 224.5     |
| 3         | -0.001751 | 0.8291    | 0.3656    | 0.5518    | 0.9348    | 413.9     |
| 4         | -0.001603 | 0.4547    | 0.3974    | 0.7851    | 0.8952    | 29.26     |
| 5         | -0.001662 | 0.8736    | 0.2202    | 0.585     | 0.828     | 492.8     |
| 6         | -0.001596 | 0.8947    | 0.7618    | 0.5987    | 0.8596    | 106.9     |
| 7         | -0.001578 | 0.5739    | 0.8524    | 0.8735    | 0.9646    | 375.2     |
| 8         | -0.001694 | 0.2344    | 0.8477    | 0.5305    | 0.6475    | 498.7     |
| 9         | -0.001587 | 0.477     | 0.943     | 0.6426    | 0.7313    | 236.9     |
| 10        | -0.001579 | 0.04052   | 0.6916    | 0.58

In [21]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [22]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'bootstrap': 0.7509273557198634, 'bootstrap_features': 0.9505830457509175, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 107.8705780889644}


In [23]:
bootstraps = [True, False]
bootstrap_feature = [True, False]

best_bs = BaggingRegressor(
    n_estimators=int(best_params['n_estimators']),
    max_samples=float(best_params['max_samples']),
    max_features=float(best_params['max_features']),
    bootstrap=bootstraps[int(best_params['bootstrap'])],
    bootstrap_features=bootstrap_feature[int(best_params['bootstrap_features'])]
)
best_bs.fit(X_train, y_train)

BaggingRegressor(bootstrap_features=True, n_estimators=107)

In [24]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.0015555101238190871
MAPE: 12588846401.65%
